<a href="https://colab.research.google.com/github/mayanksreal/marks-evaluation/blob/main/final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Handwritten Number Recognition Model (CNN)

## Importing Libraries

In [ ]:
!pip install easyocr
!pip install pymupdf
import io
import PIL
import numpy as np
import os
import cv2
import csv
from google.colab.patches import cv2_imshow
import matplotlib.pyplot as plt


from tensorflow.keras.preprocessing import image #type:ignore
from tensorflow.keras.preprocessing.image import ImageDataGenerator #type:ignore
from tensorflow.keras.datasets import mnist #type:ignore
from tensorflow.keras.models import Sequential #type:ignore
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Input #type:ignore
from tensorflow.keras.callbacks import EarlyStopping #type:ignore
from tensorflow.keras.models import load_model #type:ignore

from google.colab.patches import cv2_imshow #type:ignore
import easyocr
reader = easyocr.Reader(['en'])
import pymupdf

## Importing Dataset

In [ ]:
(x_train, y_train) , (x_test, y_test) = mnist.load_data()

## Reshaping

In [ ]:
x_train, y_train = x_train.reshape(60000, 28,28,1) , y_train.reshape(60000, 1)
x_test, y_test = x_test.reshape(10000, 28,28,1) , y_test.reshape(10000, 1)

## Normalization

In [ ]:
x_train, x_test = x_train/255.0 , x_test/255.0

## Building Model / Train

In [ ]:
try:
    model = load_model('model.keras')
except:

    model = Sequential()

    model.add(Input(shape=(28,28,1)))
    model.add(Conv2D(32, (3,3), activation = 'relu'))
    model.add(MaxPooling2D(2,2))

    model.add(Conv2D(64, (3,3), activation = 'relu'))
    model.add(MaxPooling2D(2,2))

    model.add(Flatten())
    model.add(Dense(64, activation = 'relu'))
    model.add(Dense(10, activation = 'softmax'))

    model.compile(loss = 'sparse_categorical_crossentropy' , optimizer = 'adam' , metrics = ['accuracy'])
    early = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

    model.fit(x = x_train , y = y_train, validation_split = 0.2, epochs = 30, batch_size = 1000, callbacks = [early])
    model.save('model.keras')

In [ ]:
model.evaluate(x_test, y_test)

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.4891 - loss: 1.4745


[1.4906463623046875, 0.4903999865055084]

# OCR/ CNN function

## CNN

In [ ]:
def hand_prediction(img_obj):
    image = img_obj
    def remove_borders(image, border_size=5):
        # Crop out the borders by a fixed size
        return image[border_size:-border_size, border_size:-border_size]
    # Remove borders from the entire image
    image = remove_borders(image, border_size=5)

    image = cv2.cvtColor(image, cv2. COLOR_BGR2GRAY)

    # binary thresholding
    _, thresh = cv2.threshold(image, 128, 255, cv2.THRESH_BINARY_INV)
    # Contour Detect
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    # Contour sort
    contours = sorted(contours, key=lambda ctr: cv2.boundingRect(ctr)[0])

    digits = []

    global num
    global probability
    global n
    num = 0

    for ctr in contours:
        num = num*10
        # bounding box contour
        x, y, w, h = cv2.boundingRect(ctr)

        padding = 5  # EXPERIMENTAL
        x = max(x - padding, 0)
        y = max(y - padding, 0)
        w = min(w + 2 * padding, image.shape[1] - x)
        h = min(h + 2 * padding, image.shape[0] - y)
        # Digit Extract
        digit = thresh[y:y+h, x:x+w]
        # Resizing to model input size
        resized_digit = cv2.resize(digit, (28, 28), interpolation=cv2.INTER_AREA)
        # Normalization
        digit = resized_digit / 255.0
        r = model.predict(digit.reshape(1,28,28,1))

        if np.max(r) < 0.95:

            cv2_imshow(np.reshape((digit*255), (28,28,1)))
            print(f"Predicted:{np.argmax(r)} ({round(np.max(r)*100,2)})")
            print("Actual:")
            n = input()
            if n == '':
                pass
                num = int(num/10)
            else:
              n = int(n)
              digit = digit.reshape(1,28,28,1)
              model.fit(digit, np.array([[n]]))
              model.save('model.keras')
              num = num + n
        else:
            num = num + np.argmax(r)
            print(np.max(r))


    return [num]

## OCR

In [ ]:
def ocr_prediction(img_obj):
    try:
        results = reader.readtext(img_obj)
        result_set = set()
        if len(results) > 1:
            for x in results[0][1]:
                result_set.add(x)
            for x in results[1][1]:
                result_set.add(x)

        if results:
            return results
        else:
            return 'ERR'
    except Exception as e:
        return 'ERR'

## ALL PAGES
Now for all pages, we cascadingly call all functions, in a for loop


In [ ]:
def read_pdf(pdf_path):
    pdf_document = pymupdf.open(pdf_path)
    global rows_written
    rows_written = 0

    if not os.path.exists('outputs'):
        os.makedirs('outputs')


    #Clearing any past file
    with open('output.csv', mode='w') as file:
      file.close()

    for page_number in range(len(pdf_document)):
        page = pdf_document.load_page(page_number)
        # To image (using matrix transformation)
        zoom = 2  # Adjust zoom level as needed
        mat = pymupdf.Matrix(zoom, zoom)
        pix = page.get_pixmap(matrix=mat, alpha=False)

        # pixmap to PIL Image
        img_bytes = pix.tobytes("ppm")
        image = PIL.Image.open(io.BytesIO(img_bytes))

        image = np.array(image)

        #return image

        #instead lets read all pages

        table = get_table(image, page_number)

In [ ]:
'''
def get_table(image, page_number):
    ## Filters
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    edges = cv2.Canny(blurred, 50, 150)


    ## Contour detection
    contours, _ = cv2.findContours(edges.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contours = sorted(contours, key=cv2.contourArea, reverse=True)[:1]
    table_contour = None
    for contour in contours:
        perimeter = cv2.arcLength(contour, True)
        approx = cv2.approxPolyDP(contour, 0.02 * perimeter, True)

        if len(approx) == 4:
            table_contour = approx
            break

    if table_contour is not None:
        # Get bounding box coordinates
        x, y, w, h = cv2.boundingRect(table_contour)

        # Crop the table region from the original image
        table_image = image[y:y+h, x:x+w]
        num_rows = 0

        #return table_image
        cv2_imshow(table_image)
        extract_cells_to_csv(table_image, page_number)

    else:
        print("error")
'''

def get_table(image, page_number):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    binary = cv2.adaptiveThreshold(blurred, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 11, 2)

    contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    max_contour = max(contours, key=cv2.contourArea)

    epsilon = 0.02 * cv2.arcLength(max_contour, True)
    approx = cv2.approxPolyDP(max_contour, epsilon, True)

    if len(approx) == 4:
        pts = approx.reshape(4, 2)
        rect = np.zeros((4, 2), dtype="float32")
        s = pts.sum(axis=1)
        rect[0] = pts[np.argmin(s)]
        rect[2] = pts[np.argmax(s)]
        diff = np.diff(pts, axis=1)
        rect[1] = pts[np.argmin(diff)]
        rect[3] = pts[np.argmax(diff)]

        widthA = np.linalg.norm(rect[2] - rect[3])
        widthB = np.linalg.norm(rect[1] - rect[0])
        maxWidth = max(int(widthA), int(widthB))

        heightA = np.linalg.norm(rect[1] - rect[2])
        heightB = np.linalg.norm(rect[0] - rect[3])
        maxHeight = max(int(heightA), int(heightB))

        dst = np.array([
            [0, 0],
            [maxWidth - 1, 0],
            [maxWidth - 1, maxHeight - 1],
            [0, maxHeight - 1]
        ], dtype="float32")

        M = cv2.getPerspectiveTransform(rect, dst)
        warped = cv2.warpPerspective(image, M, (maxWidth, maxHeight))
#        cv2_imshow(warped)
        extract_cells_to_csv(warped, page_number)
    else:
        x, y, w, h = cv2.boundingRect(max_contour)
        cropped = image[y:y+h, x:x+w]
        extract_cells_to_csv(cropped, page_number)


ADDING PROBABILITY BASED WARNING FOR CNN

In [ ]:
def extract_cells_to_csv(image, page_number):
    # Load the image
    height, width = image.shape[:2]

    # Convert to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Thresholding to get a binary image
    _, binary = cv2.threshold(gray, 150, 255, cv2.THRESH_BINARY_INV)

    # Find horizontal and vertical lines
    horizontal_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (25, 1))
    horizontal_lines = cv2.morphologyEx(binary, cv2.MORPH_OPEN, horizontal_kernel, iterations=2)

    vertical_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1, 25))
    vertical_lines = cv2.morphologyEx(binary, cv2.MORPH_OPEN, vertical_kernel, iterations=2)

    # Combine lines to get the grid
    grid = cv2.add(horizontal_lines, vertical_lines)

    # Find contours in the grid
    contours, _ = cv2.findContours(grid, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    # Filter out small contours and large contours (like the whole table)
    filtered_contours = []
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        if w >= 10 and h >= 10 and not (w >= width * 0.7 or h >= height * 0.8):
            filtered_contours.append((x, y, w, h))

    # Sort contours primarily by y-coordinate (to form rows) and then by x-coordinate (to form columns)
    filtered_contours.sort(key=lambda c: (c[1], c[0]))

    # Group cells by row
    rows = []
    current_row = []
    last_y = -1
    height_threshold = 15  # Set a threshold for vertical spacing to determine row separation

    for (x, y, w, h) in filtered_contours:
        if last_y == -1 or abs(y - last_y) < height_threshold:  # If in the same row
            current_row.append((x, y, w, h))
        else:  # New row detected
            if current_row:  # Append the previous row
                rows.append(current_row)
            current_row = [(x, y, w, h)]  # Start a new row

        last_y = y

    # Don't forget to add the last row
    if current_row:
        rows.append(current_row)

    file = open('output.csv', 'a')

    # Display each row's cells together
    for row in rows:
        # Sort cells in the current row by x-coordinate for display order
        row_sorted = sorted(row, key=lambda c: c[0])
        for (x, y, w, h) in row_sorted:
            cell_img = image[y:y+h, x:x+w]
            #cv2_imshow(cell_img)
            #cv2.waitKey(1000)  # Wait for 1 second to show each cell

            # For last column
            if x > max(row_sorted, key=lambda c: c[0])[0]*0.9 and y*0.9 > min(min(row, key=lambda c: c[1])[1] for row in rows) :
              file.write(str(hand_prediction(cell_img)[0]))

            #Header row
            elif (page_number != 0 and y*0.9 < min(min(row, key=lambda c: c[1])[1] for row in rows) ):
              pass

            #Other cols
            else:
              try:
                strr = ocr_prediction(cell_img)[0][1]
                if ',' in strr:
                  strr = strr.replace(',',"")
                file.write(strr)
              except Exception:
                scale_factor = 2
                upscaled = cv2.resize(cell_img, None, fx=scale_factor, fy=scale_factor, interpolation=cv2.INTER_LINEAR)
                try:
                  strr = ocr_prediction(upscaled)[0][1]
                  if ',' in strr:
                    strr = strr.replace(',',"")
                  file.write(strr)
                except Exception:
                  try:
                    scale_factor = 4
                    upscaled = cv2.resize(cell_img, None, fx=scale_factor, fy=scale_factor, interpolation=cv2.INTER_LINEAR)
                    blur = cv2.blur(upscaled, (5, 5))
                    strr = ocr_prediction(blur)[0][1]
                    if ',' in strr:
                      strr = strr.replace(',',"")
                    file.write(strr)
                  except Exception:
                    file.write("ERor")

              if x < max(row_sorted, key=lambda c: c[0])[0]*0.9:
                file.write(',')

        if not (page_number != 0 and y*0.9 < min(min(row, key=lambda c: c[1])[1] for row in rows)):
          file.write("\n")

In [ ]:
read_pdf("pdf1.pdf")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step


Predicted:1 (94.23)
Actual:
1
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - accuracy: 1.0000 - loss: 0.0594
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step


Predicted:2 (88.62)
Actual:
2
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 1.0000 - loss: 0.1208
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step


Predicted:2 (69.6)
Actual:

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step


Predicted:2 (89.9)
Actual:

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
0.99960315
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
0.987693
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
0.98197836
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step


...

Predicted:2 (40.43)
Actual:

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
0.979095
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step


Predicted:2 (76.54)
Actual:

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
0.99888223
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step


Predicted:6 (92.28)
Actual:
6
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - accuracy: 1.0000 - loss: 0.0803
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step


Predicted:8 (82.45)
Actual:
8
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 1.0000 - loss: 0.1929
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step


Predicted:2 (46.74)
Actual:

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
0.9730664
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
0.9932974
